In [2]:
text_list =  [input('\nข้อความ : ')] # รับ input

In [3]:
print(text_list)
#print(lines)

['']


In [4]:
import re
import string

def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ …
    msg = re.sub(r'…','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [5]:
print('original text:\n',text_list[0])
print('clean text:\n',clean_msg(text_list[0]))

original text:
 
clean text:
 


In [6]:
print('original text:\n',text_list[1])
print('clean text:\n',clean_msg(text_list[1]))

IndexError: list index out of range

In [142]:
clean_text = [clean_msg(txt) for txt in text_list]

In [143]:
#!pip install pythainlp
#!pip install stop_words
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words

In [144]:
import nltk
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

#print(th_stop)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Seena\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [145]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    #tokens = [i for i in tokens ]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [147]:
print('tokenized text:\n',split_word(clean_msg(text_list[0])))
#print('tokenized text:\n',split_word(clean_msg(text_list[1])))
#print('tokenized text:\n',split_word(clean_msg(text_list[2])))
#print('tokenized text:\n',split_word(clean_msg(text_list[3])))
#print('tokenized text:\n',split_word(clean_msg(text_list[4])))
#print('tokenized text:\n',split_word(clean_msg(text_list[5])))
#print('tokenized text:\n',split_word(clean_msg(text_list[6])))
#print('tokenized text:\n',split_word(clean_msg(text_list[7])))
#print('tokenized text:\n',split_word(clean_msg(text_list[8])))
#print('tokenized text:\n',split_word(clean_msg(text_list[9])))

tokenized text:
 ['สวัสดี']


In [148]:
tokens_list = [split_word(txt) for txt in clean_text]

In [149]:
tokens_list

[['สวัสดี']]

In [150]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [151]:
vocab = cvec.vocabulary_
print(tokens_list_j)

['สวัสดี']


In [152]:
c_feat[:,:20].todense()

matrix([[1]], dtype=int64)

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
t_feat = tvec.fit_transform(tokens_list_j)

In [154]:
t_feat[:,:5].todense()

matrix([[1.]])

In [155]:
print(len(tvec.idf_),len(tvec.vocabulary_))

1 1


In [156]:
c_feat[:,:5].todense()

matrix([[1]], dtype=int64)

In [157]:
print(c_feat.todense())

[[1]]


In [158]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [159]:
X = np.array(c_feat.todense())

In [160]:
X

array([[1]], dtype=int64)

In [161]:
 y = np.array(['+', '+', '-', '-','+','-','+','+','-','+'])

In [1]:
clf = MultinomialNB(class_prior=[0.25, 0.5])

# Train model
model = clf.fit(X, y)

NameError: name 'MultinomialNB' is not defined

In [310]:
print(model)

MultinomialNB(class_prior=[0.25, 0.5])


In [302]:
# Create new observation
new_observation = [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [303]:
# Predict new observation's class
y_predicted = model.predict(X)

In [304]:
# save the data to disk
data = 'y_predicted.pkl'
pickle.dump(y_predicted, open(data, 'wb'))

In [305]:
print(y_predicted)

['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+'
 '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+'
 '+' '+' '+' '+' '-' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+'
 '+' '-' '-' '+' '+' '+' '+' '+' '+' '+' '+' '+' '+' '-' '+' '+' '+' '+'
 '+' '+' '+' '+' '+' '+' '+' '-' '-' '+' '+' '+' '+